<a href="https://colab.research.google.com/github/Praxis-QR/BDSN/blob/main/KK_D3_MongoDB_Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](https://4.bp.blogspot.com/-gbL5nZDkpFQ/XScFYwoTEII/AAAAAAAAAGY/CcVb_HDLwvs2Brv5T4vSsUcz7O4r2Q79ACK4BGAYYCw/s1600/kk3-header00-beta.png)<br>


<hr>

[Prithwis Mukerjee](http://www.linkedin.com/in/prithwis)<br>

##References
https://stackoverflow.com/questions/45142629/cant-connect-to-mongo-db-via-spark <br>
https://docs.mongodb.com/spark-connector/current/python-api/

#Packages

In [1]:
from pymongo import MongoClient
import pandas as pd
import os

#Install & Test LOCAL MongoDB

In [2]:
!apt install mongodb > /dev/null
!service mongodb start



 * Starting database mongodb
   ...done.


In [3]:
mongoURI = "mongodb://localhost:27017/"
client = MongoClient(mongoURI)
client.list_database_names() # ['admin', 'local']

['admin', 'local']

In [4]:
db = client.local
#db = client.sparkDB
#db = client.people
db.list_collection_names()

['startup_log']

In [5]:
table = db.startup_log
#table = db.coll01
#table = db.contacts
#table.count_documents({}) #gives the number of documents in the table
table.find_one()

{'_id': 'ca45fa968e6e-1651036019804',
 'buildinfo': {'allocator': 'tcmalloc',
  'bits': 64,
  'buildEnvironment': {'cc': 'cc: cc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0',
   'ccflags': '-fno-omit-frame-pointer -fno-strict-aliasing -ggdb -pthread -Wall -Wsign-compare -Wno-unknown-pragmas -Wno-error=c++1z-compat -Wno-error=noexcept-type -Wno-error=format-truncation -Wno-error=int-in-bool-context -Winvalid-pch -Werror -O2 -Wno-unused-local-typedefs -Wno-unused-function -Wno-deprecated-declarations -Wno-unused-const-variable -Wno-unused-but-set-variable -Wno-missing-braces -fstack-protector-strong -fno-builtin-memcmp',
   'cxx': 'g++: g++ (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0',
   'cxxflags': '-g -O2 -fdebug-prefix-map=/build/mongodb-FmqHhf/mongodb-3.6.3=. -fstack-protector-strong -Wformat -Werror=format-security -Woverloaded-virtual -Wno-maybe-uninitialized -std=c++14',
   'distarch': 'x86_64',
   'distmod': '',
   'linkflags': '-Wl,-Bsymbolic-functions -Wl,-z,relro -pthread -Wl,-z,now -rdynami

#Connect to and Test Remote MongoDB

#Install Spark

## Deprecated - dont use these commands

In [ ]:
!apt-get update > /dev/null
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

#!wget -q http://apache.osuosl.org/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!wget -q http://apache.osuosl.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
#
# if the current version of Spark is not used, there may be errors
# check here for current versions http://apache.osuosl.org/spark
#

#!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop3.2.tgz
#!pip install -q findspark              --- no more required,  replaced by pyspark
!pip install -q pyspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

#os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop3.2"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"

     |████████████████████████████████| 281.3 MB 38 kB/s 
     |████████████████████████████████| 198 kB 55.3 MB/s 


## Simplified Install 2022

In [6]:
!pip3 install -q pyspark

     |████████████████████████████████| 281.4 MB 36 kB/s 
     |████████████████████████████████| 198 kB 61.9 MB/s 


#Spark2Mongo

In [7]:
#import findspark
#findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .master("local[*]") \
    .config("spark.mongodb.input.uri", "mongodb://localhost:27017/sparkDB.coll01") \
    .config("spark.mongodb.output.uri", "mongodb://localhost:27017/sparkDB.coll01") \
    .config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector_2.12:3.0.1')\
    .getOrCreate()

In [ ]:
#sc.stop()

In [8]:
sc = spark.sparkContext
#sc

##Write to MongoDB

In [9]:
people = spark.createDataFrame([("Bilbo Baggins",  50), ("Gandalf", 1000), ("Thorin", 195), ("Balin", 178), ("Kili", 77),
   ("Dwalin", 169), ("Oin", 167), ("Gloin", 158), ("Fili", 82), ("Bombur", None)], ["name", "age"])

In [10]:
people.show()

+-------------+----+
|         name| age|
+-------------+----+
|Bilbo Baggins|  50|
|      Gandalf|1000|
|       Thorin| 195|
|        Balin| 178|
|         Kili|  77|
|       Dwalin| 169|
|          Oin| 167|
|        Gloin| 158|
|         Fili|  82|
|       Bombur|null|
+-------------+----+



In [11]:
people.write.format("mongo").mode("append").save()

In [12]:
people.write.format("mongo").mode("append").option("database","people").option("collection", "contacts").save()

In [13]:
client.list_database_names() # ['admin', 'local']

['admin', 'local', 'people', 'sparkDB']

##Check with MongoDB client

In [14]:
client.list_database_names() # ['admin', 'local']

['admin', 'local', 'people', 'sparkDB']

In [15]:
db = client.people
list(db.list_collections())

[{'idIndex': {'key': {'_id': 1},
   'name': '_id_',
   'ns': 'people.contacts',
   'v': 2},
  'info': {'readOnly': False,
   'uuid': Binary(b'\xf7]\x86\xcf\x86\xfaM\xe8\xb1\x8a\xd9\xa3q[\xfbC', 4)},
  'name': 'contacts',
  'options': {},
  'type': 'collection'}]

In [16]:
db2 = client.sparkDB
list(db2.list_collections())

[{'idIndex': {'key': {'_id': 1},
   'name': '_id_',
   'ns': 'sparkDB.coll01',
   'v': 2},
  'info': {'readOnly': False,
   'uuid': Binary(b'\xa73\x86\xde2k@5\x85:>\xc6\xa8\xbd)Q', 4)},
  'name': 'coll01',
  'options': {},
  'type': 'collection'}]

In [17]:
kollection = db.contacts
c = kollection.find()
for crnt in c:
    print(crnt)

{'_id': ObjectId('6268d0161041830813b29efb'), 'name': 'Dwalin', 'age': 169}
{'_id': ObjectId('6268d0161041830813b29efc'), 'name': 'Oin', 'age': 167}
{'_id': ObjectId('6268d0161041830813b29efd'), 'name': 'Gloin', 'age': 158}
{'_id': ObjectId('6268d0161041830813b29efe'), 'name': 'Fili', 'age': 82}
{'_id': ObjectId('6268d0161041830813b29eff'), 'name': 'Bombur'}
{'_id': ObjectId('6268d0161041830813b29f00'), 'name': 'Bilbo Baggins', 'age': 50}
{'_id': ObjectId('6268d0161041830813b29f01'), 'name': 'Gandalf', 'age': 1000}
{'_id': ObjectId('6268d0161041830813b29f02'), 'name': 'Thorin', 'age': 195}
{'_id': ObjectId('6268d0161041830813b29f03'), 'name': 'Balin', 'age': 178}
{'_id': ObjectId('6268d0161041830813b29f04'), 'name': 'Kili', 'age': 77}


In [18]:
pandasDF = pd.DataFrame(kollection.find())
pandasDF

,_id,name,age
0,6268d0161041830813b29efb,Dwalin,169.0
1,6268d0161041830813b29efc,Oin,167.0
2,6268d0161041830813b29efd,Gloin,158.0
3,6268d0161041830813b29efe,Fili,82.0
4,6268d0161041830813b29eff,Bombur,NaN
5,6268d0161041830813b29f00,Bilbo Baggins,50.0
6,6268d0161041830813b29f01,Gandalf,1000.0
7,6268d0161041830813b29f02,Thorin,195.0
8,6268d0161041830813b29f03,Balin,178.0
9,6268d0161041830813b29f04,Kili,77.0


##Read from MongoDB

In [19]:
sparkDF = spark.read.format("mongo").option("uri","mongodb://127.0.0.1/people.contacts").load()
sparkDF.show()

+--------------------+----+-------------+
|                 _id| age|         name|
+--------------------+----+-------------+
|{6268d01610418308...| 169|       Dwalin|
|{6268d01610418308...| 167|          Oin|
|{6268d01610418308...| 158|        Gloin|
|{6268d01610418308...|  82|         Fili|
|{6268d01610418308...|null|       Bombur|
|{6268d01610418308...|  50|Bilbo Baggins|
|{6268d01610418308...|1000|      Gandalf|
|{6268d01610418308...| 195|       Thorin|
|{6268d01610418308...| 178|        Balin|
|{6268d01610418308...|  77|         Kili|
+--------------------+----+-------------+



#Chronobooks <br>
![alt text](https://1.bp.blogspot.com/-lTiYBkU2qbU/X1er__fvnkI/AAAAAAAAjtE/GhDR3OEGJr4NG43fZPodrQD5kbxtnKebgCLcBGAsYHQ/s600/Footer2020-600x200.png)<hr>
Chronotantra and Chronoyantra are two science fiction novels that explore the collapse of human civilisation on Earth and then its rebirth and reincarnation both on Earth as well as on the distant worlds of Mars, Titan and Enceladus. But is it the human civilisation that is being reborn? Or is it some other sentience that is revealing itself. 
If you have an interest in AI and found this material useful, you may consider buying these novels, in paperback or kindle, from [http://bit.ly/chronobooks](http://bit.ly/chronobooks)